<a href="https://colab.research.google.com/github/SaulHuitzil/Boolean-Networks/blob/main/Boolean_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from typing import List, Dict
import random
from typing import List, Union, Optional

In [2]:
# !pip install import-ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.2 MB/s eta 0:00:00


In [6]:
!wget -O Node.ipynb https://raw.githubusercontent.com/SaulHuitzil/Boolean-Networks/refs/heads/main/Node.ipynb

--2025-01-24 13:54:08--  https://raw.githubusercontent.com/SaulHuitzil/Boolean-Networks/refs/heads/main/Node.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307698 (300K) [text/plain]
Saving to: ‘Node.ipynb’

Node.ipynb          100%[===================>] 300.49K  --.-KB/s    in 0.03s   

2025-01-24 13:54:08 (11.2 MB/s) - ‘Node.ipynb’ saved [307698/307698]



In [9]:
import import_ipynb
from Node import Node

In [16]:
from typing import List
import random
import math


class Network:
    """
    Represents a network of nodes (`Node`), designed for modeling Boolean systems.
    """

    def __init__(self):
        # List of nodes in the network.
        self.Nodes: List[Node] = []

        # Probabilities for mutations and activations.
        self.p_add_connection = 0.5
        self.p_activate_node = 0.5
        # Random number generator for consistent randomness.
        self.random = random.Random()

    def generate_random_state(self):
        """
        Generates a random binary string representing the initial state of the network.
        """
        return ''.join(random.choice('01') for _ in range(len(self.Nodes)))

    def clone(self) -> 'Network':
        """
        Creates a deep copy of the current network, including all nodes.
        """
        cloned_network = Network()
        cloned_network.Nodes = [node.clone() for node in self.Nodes]
        return cloned_network

    def create_network(self, num_nodes: int, connectivity: float):
        """
        Initializes the network with a specified number of nodes and connectivity.
        """
        self.Nodes = []
        for _ in range(num_nodes):
            node = Node()
            node.create_entries(connectivity, num_nodes)
            #node.create_entries_with_poisson(num_nodes, connectivity)
            node.create_boolean_function()
            self.Nodes.append(node)


    def _perform_single_update(self, state: str) -> str:
        """
        Performs a single update of the network state.
        :param state: Binary string representing the current state of all nodes.
        :return: Binary string representing the updated state.
        """
        updated_state = []
        for node in self.Nodes:
            configuration = [state[entry] for entry in node.Entries]
            configuration_str = ''.join(configuration)
            updated_state.append(node.BF[configuration_str])
        return ''.join(updated_state)

    def update_state(self, initial_state: str, time_steps: Optional[int] = None) -> Union[str, List[str]]:
        """
        Updates the state of the network.
        If `time_steps` is not provided, it performs a single update.
        If `time_steps` is provided, it updates the state for the specified number of time steps.

        :param initial_state: Binary string representing the current state of all nodes.
        :param time_steps: Number of time steps to iterate (optional).
        :return: Binary string for a single update or a list of binary strings for multiple updates.
        """
        # Validation: Ensure the length of the initial state matches the number of nodes
        if len(initial_state) != len(self.Nodes):
            raise ValueError("The length of 'initial_state' must match the number of nodes in the network.")

        if time_steps is None:
            # Single update
            return self._perform_single_update(initial_state)
        else:
            # Multiple updates
            state = initial_state
            states = []
            for _ in range(time_steps):
                state = self._perform_single_update(state)
                states.append(state)
            return states





    def average_connectivity(self) -> float:
        """
        Calculates the average connectivity of the network.
        """
        return float(sum(len(node.Entries) for node in self.Nodes)) / len(self.Nodes)

    def fraction_of_ones(self) -> float:
        """
        Calculates the fraction of '1's in the Boolean functions of all nodes.
        """
        all_outputs = ''.join(''.join(node.BF.values()) for node in self.Nodes)
        return all_outputs.count('1') / len(all_outputs) if all_outputs else 0.0

    def sensitivity(self) -> float:
        """
        Estimates the sensitivity of the network based on its connectivity and Boolean function outputs.
        """
        k = self.average_connectivity()
        p = self.fraction_of_ones()
        return 2 * p * (1 - p) * k

    def mutate_conecction(self):
        """
        Applies mutations to the network by rewiring node connections.
        """
        node = random.choice(self.Nodes)
        if self.random.random() < 0.5:
            # Add a new connection.
            available_nodes = [i for i in range(len(self.Nodes)) if i not in node.Entries]
            if available_nodes:
                new_entry = self.random.choice(available_nodes)
                node.add_entry(new_entry)
        else:
            # Remove an existing connection.
            if len(node.Entries) > 1:
                entry_to_remove = self.random.choice(node.Entries)
                node.remove_entry(entry_to_remove)


    def mutate_functions(self):
        """
        Applies mutations to the network by fliping node Boolean functions values.
        """
        node = random.choice(self.Nodes)
        key = random.choice(list(node.BF.keys()))
        node.BF[key] = '1' if node.BF[key] == '0' else '0'


    def mutate(self):
      """
      Applies a random mutation to the network.
      """
      if self.random.random() < 0.5:
        self.mutate_functions()
      else:
        self.mutate_conecction()



In [20]:
if __name__ == "__main__":
    import random

    # Seed for reproducibility
    #random.seed(42)

    # Create a network
    num_nodes = 10
    connectivity = 2

    print("=== Testing Network Creation ===")
    network = Network()
    network.create_network(num_nodes, connectivity)
    print(f"Created network with {len(network.Nodes)} nodes.")
    print(f"Average connectivity: {network.average_connectivity():.2f}")
    print(f"Fraction of ones: {network.fraction_of_ones():.2f}")
    print(f"Sensitivity: {network.sensitivity():.2f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")
    state0 = network.generate_random_state()
    print(f"Initial state: {state0}")

    # Test state update
    print("\n=== Testing State Update ===")
    state = state0
    for t in range(10):
      state = network.update_state(state)
      print(f"state_{t}: {state}")

    # Test states for num_time_steps
    print("\n=== Testing States for num_time_steps ===")
    num_time_steps = 5
    states = network.update_state(state0, num_time_steps)
    for t, state in enumerate(states):
        print(f"state_{t}: {state}")

    # Clone the network
    print("\n=== Testing Network Cloning ===")
    cloned_network = network.clone()
    print("Cloned network created.")
    print(f"Cloned average connectivity: {cloned_network.average_connectivity():.5f}")
    print(f"Fraction of ones: {cloned_network.fraction_of_ones():.5f}")
    print(f"Sensitivity: {cloned_network.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(cloned_network.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")

    # Mutate the network - Connection
    print("\n=== Testing Network Mutation - Connection ===")
    network_mutated_connection = network.clone()
    network_mutated_connection.mutate_conecction()
    print(f"Post-mutation average connectivity: {network_mutated_connection.average_connectivity():.5f}")
    print(f"Fraction of ones: {network_mutated_connection.fraction_of_ones():.5f}")
    print(f"Sensitivity: {network_mutated_connection.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network_mutated_connection.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")


    # Mutate the network - Boolean Function
    print("\n=== Testing Network Mutation - Boolean Function ===")
    network_mutated_function = network.clone()
    network_mutated_function.mutate_functions()
    print(f"Post-mutation average connectivity: {network_mutated_function.average_connectivity():.5f}")
    print(f"Fraction of ones: {network_mutated_function.fraction_of_ones():.5f}")
    print(f"Sensitivity: {network_mutated_function.sensitivity():.5f}")
    print("Nodes and their connections:")
    for idx, node in enumerate(network_mutated_function.Nodes):
        print(f"  Node {idx}: Entries={node.Entries}, BF={node.BF}")


=== Testing Network Creation ===
Created network with 10 nodes.
Average connectivity: 2.00
Fraction of ones: 0.50
Sensitivity: 1.00
Nodes and their connections:
  Node 0: Entries=[8, 2], BF={'00': '1', '01': '0', '10': '1', '11': '0'}
  Node 1: Entries=[1, 2], BF={'00': '1', '01': '0', '10': '1', '11': '1'}
  Node 2: Entries=[0, 4], BF={'00': '1', '01': '0', '10': '0', '11': '0'}
  Node 3: Entries=[0, 5], BF={'00': '0', '01': '0', '10': '1', '11': '1'}
  Node 4: Entries=[9, 7], BF={'00': '0', '01': '0', '10': '1', '11': '1'}
  Node 5: Entries=[1, 7], BF={'00': '1', '01': '1', '10': '0', '11': '1'}
  Node 6: Entries=[9, 7], BF={'00': '0', '01': '1', '10': '1', '11': '0'}
  Node 7: Entries=[6, 0], BF={'00': '0', '01': '0', '10': '0', '11': '1'}
  Node 8: Entries=[5, 0], BF={'00': '1', '01': '1', '10': '1', '11': '0'}
  Node 9: Entries=[8, 2], BF={'00': '0', '01': '0', '10': '0', '11': '1'}
Initial state: 1111100010

=== Testing State Update ===
state_0: 0101000011
state_1: 1110101010
sta